# Analyzing data

In [10]:
import dol 

s = dol.JsonFiles('/Users/thorwhalen/Dropbox/_odata/finance/mood/news/searches')
len(s)

285

In [13]:
import pandas as pd 


headlines_store = dol.JsonFiles('/Users/thorwhalen/Dropbox/_odata/finance/mood/news/searches/yahoo_finance_headlines')
headlines = pd.DataFrame(headlines_store.values())
print(f"{headlines.shape=}")
headlines.iloc[0]

headlines.shape=(92, 66)


0     Exclusive-Ukraine bets on US LNG as gas import...
1     Alibaba Releases AI Model That Reads Emotions ...
2     German Yield Nears 3% Milestone on Historic Pi...
3     Bolivia’s New State Gold Trader Eyes $1 Billio...
4     Jackson Square Aviation orders 50 A320neo fami...
                            ...                        
61                                                 None
62                                                 None
63                                                 None
64                                                 None
65                                                 None
Name: 0, Length: 66, dtype: object

In [15]:
newsdata_store = dol.JsonFiles('/Users/thorwhalen/Dropbox/_odata/finance/mood/news/searches/newsdata')
newsdata = pd.DataFrame(newsdata_store.values())
print(f"{newsdata.shape=}")
newsdata.iloc[0]

newsdata.shape=(96, 0)


Series([], Name: 0, dtype: float64)

In [16]:
list(newsdata_store)

['2025-03-12/2025-03-12--12-00-20____data_financial_news_search_words_json.json',
 '2025-03-12/2025-03-12--06-00-17____data_financial_news_search_words_json.json',
 '2025-03-12/2025-03-12--00-00-38____data_financial_news_search_words_json.json',
 '2025-02-25/2025-02-25--18-00-22____data_inancial_news_search_words_json.json',
 '2025-02-25/2025-02-25--00-00-50____data_inancial_news_search_words_json.json',
 '2025-02-25/2025-02-25--12-00-23____data_inancial_news_search_words_json.json',
 '2025-02-25/2025-02-25--06-00-19____data_inancial_news_search_words_json.json',
 '2025-02-22/2025-02-22--18-00-18____data_inancial_news_search_words_json.json',
 '2025-02-22/2025-02-22--06-00-22____data_inancial_news_search_words_json.json',
 '2025-02-22/2025-02-22--12-00-23____data_inancial_news_search_words_json.json',
 '2025-02-22/2025-02-22--00-00-49____data_inancial_news_search_words_json.json',
 '2025-02-23/2025-02-23--18-00-20____data_inancial_news_search_words_json.json',
 '2025-02-23/2025-02-23--

In [ ]:
from mood.tools import search_and_save_news, DFLT_STORE; t = search_and_save_news('APPL', source='newsdata'); print(DFLT_STORE); print(t);

In [27]:
s = dol.Files(DFLT_STORE)
list(s)

['yahoo_finance_headlines/2025-02-19/2025-02-19--09-52-32__.json',
 'newsdata/2025-03-12/2025-03-12--18-26-25__APPL.json']

In [28]:
DFLT_STORE

'/Users/thorwhalen/.config/mood/news/searches'

'/Users/thorwhalen/Dropbox/py/proj/misc/infinvest/mood/misc'

# Updating the local data with server data

In [2]:
from xdol import update_with_policy
from xdol.updating import KeyDecision


def today_utc_date_str():
    import datetime

    today = datetime.datetime.now().date()
    return str(today)


def update_missing_or_today(target, source, *, keys_to_consider=None, verbose=True):
    """
    Update target with source, but only for keys that are missing in target or
    that contain today's date.
    """
    today = today_utc_date_str()

    def _missing_or_today_decider(key, target_info, source_info) -> KeyDecision:
        if source_info is None:
            return "SKIP"
        elif target_info is None or today in str(key):
            return "COPY"

        return "SKIP"

    return update_with_policy(
        target,
        source,
        policy=_missing_or_today_decider,
        keys_to_consider=keys_to_consider,
        verbose=verbose,
    )

In [3]:
from sshdol import SshFiles
import dol 

src_rootdir = '/root/.config/mood/news/searches'
src = SshFiles('thorwhalen', rootdir=src_rootdir, max_levels=None)

target_rootdir = '/Users/thorwhalen/Dropbox/_odata/finance/mood/news/searches'
target = dol.Files(target_rootdir)

In [4]:
update_missing_or_today(target, src, verbose=True)

yahoo_finance_headlines/2025-02-19/: COPY


KeyError: "[Errno 21] Is a directory: '/Users/thorwhalen/Dropbox/_odata/finance/mood/news/searches/yahoo_finance_headlines/2025-02-19/'"

In [12]:
from mood.tools import search_and_save_news

my_store = dict()
t = search_and_save_news("~/tmp/example.json", source="newsdata", store=my_store)
list(my_store)

['newsdata/2025-03-13/2025-03-13--14-55-34__APPL.json']

['2025-03-03/',
 '2025-03-02/',
 '2025-02-20/',
 '2025-03-01/',
 '2025-02-19/',
 '2025-02-25/',
 '2025-03-12/',
 '2025-03-04/',
 '2025-03-11/',
 '2025-03-05/',
 '2025-03-08/',
 '2025-03-07/',
 '2025-03-09/',
 '2025-03-06/',
 '2025-02-24/',
 '2025-02-26/',
 '2025-02-21/',
 '2025-02-28/',
 '2025-02-27/',
 '2025-03-10/',
 '2025-02-22/',
 '2025-02-23/']

In [ ]:
# a function that gives me, from a list of strings in the form
# '2020-01-01', '2020-01-02', etc, the one that corresponds to 
# today's UTC date

def today_utc_date_str():
    import datetime
    today = datetime.datetime.now().date()
    return str(today)

def todays_date_folder(dates):
    today_date_folder = today_utc_date_str() + '/'
    if today_date_folder in dates:
        return [today_date_folder]
    else:
        return []

def update_missing_date_folders(src: dict, target: dict):
    src_keys = set(src)
    # remove today's date folder from the target keys (if it's there)
    # since the day is not over, there may be additional data
    target_keys = set(target) - set([today_utc_date_str + '/']) 
    missing_keys = sorted(src_keys - target_keys)
    
    for missing_key in missing_keys:
        target[missing_key].update(src[missing_key])


    

'2025-03-12/'